In [ ]:
import polars as pl
import polars_ds as pds
print(pds.__version__)

In [ ]:
import numpy as np
import statsmodels.api as sm

nsample = 50
x = np.linspace(0, 20, nsample)
X = np.column_stack((x, (x - 5) ** 2))
X = sm.add_constant(X)
beta = [5.0, 0.5, -0.01]
sig = 0.5
w = np.ones(nsample)
w[nsample * 6 // 10 :] = 3
y_true = np.dot(X, beta)
e = np.random.normal(size=nsample)
y = y_true + sig * w * e
X = X[:, [0, 1]]

In [ ]:
mod_wls = sm.WLS(y, X, weights=1.0 / (w ** 2))
res_wls = mod_wls.fit()
print(res_wls.summary())

In [ ]:
df = pl.from_numpy(X, schema = ['const', 'x']).with_columns(
    y = pl.Series(name = "y", values = y),
    w = pl.Series(name = "w", values = 1.0 / (w ** 2)),
)
df

In [ ]:
df.select(
    pds.query_lstsq_report(
        "x", "const",
        target = "y",
        weights = "w",
    )
).unnest("w")

In [ ]:
df.select(
    pds.query_wls_ww(
        "x", "const",
        target = "y",
        weights = "w"
    )
)

In [ ]:
df.select(
    pds.query_wls_ww(
        "x", "const",
        target = "y",
        weights = "w",
        return_pred = True
    )
)

In [ ]:
size = 1000
df = pds.frame(size = size).select(
    pds.random(0., 1.).alias("x1"),
    pds.random(0., 1.).alias("x2"),
    pds.random(0., 1.).alias("x3"),
).with_row_index().with_columns(
    y = pl.col("x1") + pl.col("x2") * 0.2 - 0.3 * pl.col("x3")
)

window_size = 5
min_valid_rows = 3

result = df.with_columns(
    pds.query_rolling_lstsq(
        "x1", "exp(x2)", "x3",
        target = "y",
        window_size = window_size,
        min_valid_rows = min_valid_rows,
        null_policy = "skip" 
    ).alias("test")
).with_columns(
    pl.col("test").is_null().alias("is_null")
)

In [ ]:
df.select(
    "x1",
    pl.col("x1").shift(1).alias("x2")
).select(
    x1 = pl.col("x1") - pl.col("x2")
    , x2 = pl.col("x1").diff()
)

In [ ]:
df = pl.DataFrame({
    "a": [1,2,3,None]
})

df.select(
    pl.col("a").mean()
)